# Water flux within the root - Numerical solutions in DuMu${}^x$

_by Daniel Leitner, Andrea Schnepf, Timo Koch_

In the following we will give some hints how to solve Benchmarks 1-3, using [DuMu${}^x$](www.dumux.org). Scripts for the following benchmarking examples are available on [Github](https://github.com/Plant-Root-Soil-Interactions-Modelling/dumux-rosi.git), in the folder dumux-rosi/rosi_benchmarking/rootsystem/ run the scripts /python/dumux_b1.py, /python/dumux_b2.py, or /python/dumux_b3.py.

To run the examples you will need: dune (releases/2.6), dune-foamgrid (releases/2.6), dumux (releases/3.0), CRootBox (master)

## Theory

We use DuMu${}^x$ single phase Darcy flow model, which uses Darcy flow: 

\begin{eqnarray} 
v = - \frac{K}{\mu} \left( \frac{\partial p_r}{\partial z} - \rho g \right), \label{darcy}
\end{eqnarray}

where $v$ is the fluid velocity, $K$ permeability, $\mu$ the dynamic viscosity, $p_r$ the fluid pressure, $\rho$ is the fluid density, amnd $g$ the gravitational acceleration.

For incompressible fluids conservation of mass leads to 

\begin{eqnarray} 
q = div  \left( \rho \frac{K}{\mu} ( \frac{\partial p_r}{\partial z} - \rho g ) \right). \label{consdarcy}
\end{eqnarray}

Defining $q$ as the root radial surface flux yields the desired model.

## Compile program

Compile the program by calling 

*make rootsystem* 

for the following benchmarks in the folder dumux-rosi/build-cmake/rosi_benchmarking/rootsystem

Additionally, you can compile

*make rootsystem_rb*

for to calculate fluxes within a CRootBox root system. The main files will be merged into one file at later point (todo).

## Run benchmarks
All benchmarks can be simulated by just changing the Dumux .input files. The specific benchmarks can be started by Python scripts located in the folder dumux-rosi/rosi_benchmarking/rootsystem/python, the roots (even the straigth single root) are given per .dgf file in the folder dumux-rosi/rosi_benchmarking/rootsystem/grids.

## Input files

The input files for benchmarks 1-3 are located in /rootsystem/input/. The following paramteters are used:

**[Problem]**

*Name*, name of the simulation

**[ TimeLoop ]**

*TEnd*, simulation time [s]. If the value is zero a steady state problem is calculated

*DtInitial*, initial time step [s]

*MaxTimeStepSize*, maximal time step size [s]. Optionally, an not used since the benchmarks are static problems.

*CheckTimes*, optionally times [s] can be defined, when the .vtp output is written. Per default a file at start and at the final time is written.


**[RootSystem.Grid]**

*File*, name of the .dgf file, or the CRootBox parameter file (in case you run rootsystem_rb)

*InitialT*, in case of a CRootBox file name you have to state the initial growth time.

*Grow*, enables growth in case of a CRootBox file (False per default)


**[RootSystem.Conductivity]**

*Kr*, the root radial conductivity [cm / hPa /day]. Constant, per root order/type, by linear interpolation, or by multiple linear interpolation 

*KrAge*, root age [days]. Optionally, ascending values are given for linear interpolation, sample points are (*KrAge*, *Kr*).

*Kx*, the axial conductivity [cm$^4$ / hPa /day].Constant, per root order/type, by linear interpolation, or by multiple linear interpolation 

*KxAge*, root age [days]. Optionally, ascending values are given for linear interpolation, sample points are (*KxAge*, *Kx*).


**[RootSystem.Collar]**

Either state P for Dirichlet or Transpiration for Neumann boundary conditions

*P*, pressure [cm] Constant, or periodic (by stating 2 values min and max, and not RootSystem.Collar.PT), or by linear interpolation.

*PT*, simulation time [day]. Optionally, ascending values are given for linear interpolation, sample points are (*PT*, *P*).

*Transpiration*, root transpiration [kg / day] Constant, or periodic (by stating 2 values min and max, and not RootSystem.Collar.TranspirationT), or by linear interpolation.

*TranspirationT*, simulation time [day]. Optionally, ascending values are given for linear interpolation, sample points are (*TranspirationT*, *Transpiration*). 


**[Soil.IC]**

*Pressure*, the surrounding soil pressure [cm] for computation of the radial flow. If it is one value: constant pressure head is assumed. If there are more values, and additionally, *Z* values are defined, tabular linear interpolated data is assumed. If there are more values, but no *Z* values, the pressure is assumed to be given constant per root order, or type.

*Z*, z-coordinates [m]. Optionally, ascending values are given for linear interpolation, sample points are (*Z*, *Pressure*). 

** [Vtk] ** 

Some output options:

*AddProcessRank*  = "false", always false, since FoamGrid does not support parallel execution. 

*AddVelocity* = "true", it makes sense to plot the axial velocities.


## Remarks

The .dgf files for the benchmarks are created by the Python script dgf.py in the folder rootsystem/python/.

### Benchmark 1 

The dgf contains only geometry, parameters are all constant and passed via the input file. 

The Python file dumux_b1 solves Benchmark 1. Additionally, it solves for a horizontal root, and with constant transpiration. All results are compared to the analytical solutions

### Benchmark 2 

The dgf contains most parameters. Only soil pressure, and root collar pressure is given within the input file. 

### Benchmark 3

The dgf contains most parameters. Only soil pressure, and root collar pressure is given within the input file. 


## Implementation

In the following we describe the code, that is used to solve Benchmarks 1-3.

In DuMu${}^x$ we base the problem on the one-phase porous medium model (`porousmediumflow/1p/model.hh`) meaning there is one fluid phase (water) inside a porous medium (root). Hence, the root is modeled as a porous medium, where the root xylem is the pore space filled with water and all other root components constitute the solid matrix. The type of model is chosen by including the right header file in the problem file, with ```#include <dumux/porousmediumflow/1p/model.hh>```. Additionally, the spatial parameter class is derived from a suitable base class, i.e. ```FVSpatialParamsOneP```.

### main file: rootsystem.cc, rootsystem_rb.cc (todo merge into one file)

Compilation with make rootsystem, or make rootsystem_rb. Rootsystem uses a dgf grid file as geometry, while rootsystem_rb uses a static or growing root architecture generated by CRootBox. The main file is a typical Dumux 3.0 main file with the following extensions:

1. We use TimeLoop.CheckTimes and only write vtk files at the check times

2. The model is static and we always build a static model. But we use the simulation loop to use dynamic boundary conditions like a sinusoidal transpiration rate.

3. In case of a growing root system, we need to adjust the dune-foamgrid geometry, and update the spatial parameters. 

### problem file: rootsproblem.hh

A Dumux Problem class describes initial conditions, boundary conditions, and sources and sinks.

First, the constructor reads all relevant parameters from the input file, in our case: Soil.IC.P, RootSystem.Collar.Transpiration or RootSystem.Collar.P; and optionally Soil.IC.Z, and RootSystem.Collar.TranspirationT or RootSystem.Collar.PT respectively.

*Definition of boundary conditions:*

We have to choose the type of Dumux boundary condition in the member function boundaryTypesAtPos(...), and set it to either Neumann or Dirichlet. We make this decision based on if pressure or transpiration was defined in the input file. 

Dirichlet boundary conditions can not be solution dependent. They are used for constant pressure top or bot (see dirichletAtPos(...)).  

Neumann boundary conditions can be solution dependent. We use this to limit the flow hat the critical collar pressure cannot be exceeded. 

*Definition of initial condition* 

Performed with the method initial(...). You can check in first vtk, that is written, if initial data are correct.

*Sources and sinks*

They are defined by overwriting the method source(...). We implement the gradient between root pressure and soil pressure. Soil pressure is determined by a callback function (so we can use this implementation for sequential coupling at later time)  


### spatial parameters: rootspatialparams_dgf.hh

This class manages the spatial parameters if a dune grid fild (.dgf) is used. The constructor reads all relevant parameters from the input file.All parameters are optional (since they could be passed via the dgf file). RootSystem.Radius and RootSystem.Age can be constant or given per root order. Root system conductivities can be given constant, per root order/type, by linear interpolation (age dependent), or by multiple tables of linear interpolation chosen per root order/type (see also dumux/io/inputfilefunciton.hh). 

If a parameter is not defined in the input file it is assumed to be in the dgf file. In this case parameter indices are: orderIdx = 0 (root order or root type), radiusIdx = 1, ageIdx = 2, krIdx = 3, kxIdx = 4. To copy the grid data we must call initParameters(const GridData& gridData) after constructing the dune grid from the .dgf file (this is done in the main file).

porosityAtPos(...) must be overwritten, but model solution is independent of it.

permeability(...) defines the (intrinsic) permeability $K$ [m^2] and must be overwritten. We convert the root axial conductivity ($\frac{m^5 s}{kg} = \frac{m^4}{Pa \ s} $) to the intrinsic permeability by multiplication with the dynamic viscosity ($Pa \ s$) and division by the cross-section area $\pi r^2$ ($m^2$). The intrinsic permeability is a resistance coefficient that is only dependent on the root xylem and not the properties of the fluid (water).

### spatial parameters: rootspatialparams_rb.hh

This class manages the spatial parameters if a root growth model  is used (currently only CRootBox, later also CPlantBox). The constructor reads all relevant parameters from the input file, these are root system conductivities RootSystem.Conductivity.Kr and Kx, optionally KxAge and KrAge (see also dumux/io/inputfilefunciton.hh). Additionally, the initial root system simulation time must be stated RootSystem.Grid.InitialT.

To keep track of the parameter the method updateParameters(const RootSystem& rs) must be called initially, and after each time the root system grew.

porosityAtPos(...), and permeability(...) is defined as in rootspatialparams_dgf.hh.
